<a href="https://colab.research.google.com/github/Sandwhaletree/Mai_Finlab/blob/main/A_%E7%94%9F%E6%B4%BB%E6%87%89%E7%94%A8/G1_linebot_flask_ngrok_230919.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
安裝套件
'''
!pip install line-bot-sdk flask flask_ngrok

!pip install pyngrok

!ngrok authtoken 2QrnI14qFzzpIcHDa1q6PzJmPom_3wLXxhnp74vnEtaPi3ESJ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.27.1, but you have requests 2.28.2 which 

In [ ]:
# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

from linebot.models import MessageEvent, TextMessage,TextSendMessage, ImageSendMessage, StickerSendMessage, LocationSendMessage, QuickReply, QuickReplyButton, MessageAction

In [ ]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")


# 生成實體物件
line_bot_api = LineBotApi("fpNQNqeH9ztWIZQ3ixeplwEd8ri51rkX0wMTrFyJXdaYeRYga0UhRM++DYd8jK9rqvC5n3HVTdOX9M43BjMb5Nu5uPgBd9+6OITRP9cYNo2T5oNcq1a+giqzfFVO+L1bdwwYSeswfjLQodnkL1VK0wdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("10943ad57e5ea68e7604927bab7dbe3a")
run_with_ngrok(app)

In [ ]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    mtext = event.message.text
    if mtext == '@確診後流程':
        try:
            message = TextSendMessage(
                text = "確診後該怎麼辦?(SOP圖示)"
            )
            line_bot_api.reply_message(event.reply_token, message)
        except:
            line_bot_api.reply_message(event.reply_token,
                TextSendMessage(text= 'Sorry~屁桃故障囉！'))
    elif mtext == '@想看西藥＆拿西藥':
        try:
            message = ImageSendMessage(
                original_content_url = "https://ithelp.ithome.com.tw/upload/images/20220925/20151681EaMkK6ROvq.jpg",
                preview_image_url = "https://ithelp.ithome.com.tw/upload/images/20220925/20151681EaMkK6ROvq.jpg"
            )
            line_bot_api.reply_message(event.reply_token, message)
        except:
            line_bot_api.reply_message(event.reply_token,
                TextSendMessage(text= 'Sorry~屁桃故障囉！'))
    elif mtext == '@想看中藥＆拿中藥':
        try:
            message = StickerSendMessage(
                package_id = '8522',
                sticker_id = '16581289'
            )
            line_bot_api.reply_message(event.reply_token, message)
        except:
            line_bot_api.reply_message(event.reply_token,
                TextSendMessage(text= 'Sorry~屁桃故障囉！'))

In [ ]:
# 主程序運行
port = 5000
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))
app.run()

 * ngrok tunnel "https://dbbf-34-32-158-124.ngrok-free.app" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Jun/2023 00:37:08] "HEAD /https://dbbf-34-32-158-124.ngrok-free.app/ HTTP/1.1" 404 -


 * Running on http://dbbf-34-32-158-124.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
